# Mandatory Challenge
## Context
You work in the data analysis team of a very important company. On Monday, the company shares some good news with you: you just got hired by a major retail company! So, let's get prepared for a huge amount of work!

Then you get to work with your team and define the following tasks to perform:   
1. You need to start your analysis using data from the past.  
2. You need to define a process that takes your daily data as an input and integrates it.  

You are in charge of the second part, so you are provided with a sample file that you will have to read daily. To complete you task, you need the following aggregates:
* One aggregate per store that adds up the rest of the values.
* One aggregate per item that adds up the rest of the values.

You can import the dataset `warehouse_and_retail_sales` from Ironhack's database. 

## Your task
Therefore, your process will consist of the following steps:
1. Read the sample file that a daily process will save in your folder. 
2. Clean up the data.
3. Create the aggregates.
4. Write three tables in your local database: 
    - A table for the cleaned data.
    - A table for the aggregate per supplier.
    - A table for the aggregate per item.

## Instructions
* Read the csv you can find in Ironhack's database.
* Clean the data and create the aggregates as you consider.
* Create the tables in your local database.
* Populate them with your process.

In [1]:
# your code here
import numpy as np
import pandas as pd
import requests
from io import StringIO
from collections import Counter
import pymysql
from sqlalchemy import create_engine

In [2]:
orig_url = 'https://drive.google.com/file/d/1ZsHSCYciWkUd8y9mvEV5RZQiO1Qk0WLh/view?usp=sharing'

file_id = orig_url.split('/')[-2]
dwn_url = 'https://drive.google.com/uc?export=download&id=' + file_id
url = requests.get(dwn_url).text
csv_raw = StringIO(url)
raw_data = pd.read_csv(csv_raw)
raw_data.head()

,YEAR,MONTH,SUPPLIER,ITEM CODE,ITEM DESCRIPTION,ITEM TYPE,RETAIL SALES,RETAIL TRANSFERS,WAREHOUSE SALES
0,2017,4,ROYAL WINE CORP,100200,GAMLA CAB - 750ML,WINE,0.0,1.0,0.0
1,2017,4,SANTA MARGHERITA USA INC,100749,SANTA MARGHERITA P/GRIG ALTO - 375ML,WINE,0.0,1.0,0.0
2,2017,4,JIM BEAM BRANDS CO,10103,KNOB CREEK BOURBON 9YR - 100P - 375ML,LIQUOR,0.0,8.0,0.0
3,2017,4,HEAVEN HILL DISTILLERIES INC,10120,J W DANT BOURBON 100P - 1.75L,LIQUOR,0.0,2.0,0.0
4,2017,4,ROYAL WINE CORP,101664,RAMON CORDOVA RIOJA - 750ML,WINE,0.0,4.0,0.0


In [3]:
# Check if there are null values
raw_data.isnull().sum()

YEAR                 0
MONTH                0
SUPPLIER            24
ITEM CODE            0
ITEM DESCRIPTION     0
ITEM TYPE            1
RETAIL SALES         0
RETAIL TRANSFERS     0
WAREHOUSE SALES      0
dtype: int64

In [4]:
# Check why they may be null. Should they be removed or replaced?
check = raw_data.loc[raw_data['SUPPLIER'].isnull() == True]
check
'''These seems to be some internal technical records irrelevant to sales. In reall life I would ask owner of this data for
clarification but now I have to assume I'm right. This means I can remove these lines from the table.'''
clean_data = raw_data.loc[raw_data['SUPPLIER'].isnull() == False]
clean_data.head()

,YEAR,MONTH,SUPPLIER,ITEM CODE,ITEM DESCRIPTION,ITEM TYPE,RETAIL SALES,RETAIL TRANSFERS,WAREHOUSE SALES
0,2017,4,ROYAL WINE CORP,100200,GAMLA CAB - 750ML,WINE,0.0,1.0,0.0
1,2017,4,SANTA MARGHERITA USA INC,100749,SANTA MARGHERITA P/GRIG ALTO - 375ML,WINE,0.0,1.0,0.0
2,2017,4,JIM BEAM BRANDS CO,10103,KNOB CREEK BOURBON 9YR - 100P - 375ML,LIQUOR,0.0,8.0,0.0
3,2017,4,HEAVEN HILL DISTILLERIES INC,10120,J W DANT BOURBON 100P - 1.75L,LIQUOR,0.0,2.0,0.0
4,2017,4,ROYAL WINE CORP,101664,RAMON CORDOVA RIOJA - 750ML,WINE,0.0,4.0,0.0


In [5]:
# There was also one line where Item Type column had null value. Let's check if it's still there and what we can do about it.
check = clean_data[clean_data['ITEM TYPE'].isnull() == True]
check

,YEAR,MONTH,SUPPLIER,ITEM CODE,ITEM DESCRIPTION,ITEM TYPE,RETAIL SALES,RETAIL TRANSFERS,WAREHOUSE SALES
66439,2017,10,REPUBLIC NATIONAL DISTRIBUTING CO,347939,FONTANAFREDDA BAROLO SILVER LABEL 750 ML,NaN,0.0,0.0,1.0


In [6]:
'''I know that Barolo is a wine but it's also very easy to find this information in Google.'''
clean_data.loc[clean_data['ITEM TYPE'].isnull(), 'ITEM TYPE'] = 'WINE'
check = clean_data[clean_data['ITEM TYPE'].isnull() == True]
check

C:\Users\gruzd\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,YEAR,MONTH,SUPPLIER,ITEM CODE,ITEM DESCRIPTION,ITEM TYPE,RETAIL SALES,RETAIL TRANSFERS,WAREHOUSE SALES


In [7]:
print(len(clean_data['ITEM CODE'].unique()))
print(len(clean_data['ITEM DESCRIPTION'].unique()))
print(len(clean_data['ITEM CODE'].unique()) - len(clean_data['ITEM DESCRIPTION'].unique()))
'''There are some items with the same code but different description'''
check = clean_data[['ITEM CODE', 'ITEM DESCRIPTION']]
check = check.groupby('ITEM CODE')['ITEM DESCRIPTION'].nunique()
item_code_list = check[check > 1].index.values.tolist()
item_code_list

23553
23621
-68


['10444',
 '10491',
 '10636',
 '11088',
 '11373',
 '11562',
 '11584',
 '11592',
 '11838',
 '12134',
 '12227',
 '12375',
 '12475',
 '12646',
 '12769',
 '12785',
 '13374',
 '14737',
 '181226',
 '202410',
 '22318',
 '22921',
 '23146',
 '232492',
 '232934',
 '23528',
 '23772',
 '23805',
 '24020',
 '25555',
 '25561',
 '25961',
 '25962',
 '26282',
 '26516',
 '26841',
 '27817',
 '28226',
 '28290',
 '28331',
 '28380',
 '28462',
 '28490',
 '28533',
 '28616',
 '28649',
 '28728',
 '28741',
 '28762',
 '28770',
 '28797',
 '28851',
 '28927',
 '29017',
 '29033',
 '29050',
 '29068',
 '29147',
 '29203',
 '29394',
 '29416',
 '29530',
 '29622',
 '297468',
 '300250',
 '300644',
 '300992',
 '304922',
 '304986',
 '307412',
 '30796',
 '308170',
 '30870',
 '30878',
 '30881',
 '309762',
 '31051',
 '31056',
 '31060',
 '31069',
 '31095',
 '31098',
 '311080',
 '311120',
 '31149',
 '312298',
 '313115',
 '313879',
 '313882',
 '314006',
 '317169',
 '319177',
 '322695',
 '322923',
 '326504',
 '328745',
 '332039',
 '3

In [8]:
check_dict = {}
for item in item_code_list:
    lst = clean_data.loc[clean_data['ITEM CODE'] == item]['ITEM DESCRIPTION'].unique().tolist()
    check_dict[item] = lst
'''The reason for having multiple descriptions for one code is various spelling, hense it can be aligned.
I will take first spelling option by default.'''

'The reason for having multiple descriptions for one code is various spelling, hense it can be aligned.\nI will take first spelling option by default.'

In [9]:
codes = clean_data['ITEM CODE'].unique().tolist()
len(codes) # Self check that number of unique codes haven't changed since cell 7
code_dic = {}
for code in codes:
    code_dic[code] = clean_data.loc[clean_data['ITEM CODE'] == code]['ITEM DESCRIPTION'].tolist()[0]
code_dic

{'100200': 'GAMLA CAB - 750ML',
 '100749': 'SANTA MARGHERITA P/GRIG ALTO - 375ML',
 '10103': 'KNOB CREEK BOURBON 9YR - 100P - 375ML',
 '10120': 'J W DANT BOURBON 100P - 1.75L',
 '101664': 'RAMON CORDOVA RIOJA - 750ML',
 '101680': 'MANISCHEWITZ CREAM WH CONCORD - 1.5L',
 '101753': 'BARKAN CLASSIC PET SYR - 750ML',
 '10197': 'KNOB CREEK BOURBON 9YR - 100P - 1.75L',
 '101974': 'CH ST MICH P/GRIS - 750ML',
 '102083': 'CH DE LA CHESNAIE MUSCADET - 750ML',
 '10243': 'OLD GRAND DAD BOURBON 100P - 750ML',
 '10246': 'HILLROCK DOUBLE CASK RYE - 750ML',
 '10266': 'LA CHAYA MAL - 750ML',
 '10268': 'LA CHAYA TORRONTES - 750ML',
 '10269': 'FEW BOURBON - 750ML',
 '10270': 'KOPPER KETTLE WHISKEY - 750ML',
 '10271': 'ALLAGASH SAISON 6/4 NR 12.0Z',
 '10274': 'KINSEY BOURBON - 750ML',
 '10275': 'LOT 40 RYE - 750ML',
 '10306': 'TROEGS SOLID SENDER PALE ALE - 4/6NR',
 '10332': 'GREY GOOSE VODKA - 375ML',
 '103608': 'VAJRA LANGHE NEBBIOLO DOC - 750ML',
 '10370': 'BALTIMORE BEER WORKS DARK USHER - 4/6NR',
 '

In [10]:
len(code_dic) # Self check again

23553

In [11]:
clean_data['ITEM DESCRIPTION'] = clean_data['ITEM CODE'].map(code_dic)
clean_data.head()
# I also ran cell 7 to check that now item_code_list is empty as there are no multiple descriptions per one code anymore.

<ipython-input-11-73f11de231dc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_data['ITEM DESCRIPTION'] = clean_data['ITEM CODE'].map(code_dic)


,YEAR,MONTH,SUPPLIER,ITEM CODE,ITEM DESCRIPTION,ITEM TYPE,RETAIL SALES,RETAIL TRANSFERS,WAREHOUSE SALES
0,2017,4,ROYAL WINE CORP,100200,GAMLA CAB - 750ML,WINE,0.0,1.0,0.0
1,2017,4,SANTA MARGHERITA USA INC,100749,SANTA MARGHERITA P/GRIG ALTO - 375ML,WINE,0.0,1.0,0.0
2,2017,4,JIM BEAM BRANDS CO,10103,KNOB CREEK BOURBON 9YR - 100P - 375ML,LIQUOR,0.0,8.0,0.0
3,2017,4,HEAVEN HILL DISTILLERIES INC,10120,J W DANT BOURBON 100P - 1.75L,LIQUOR,0.0,2.0,0.0
4,2017,4,ROYAL WINE CORP,101664,RAMON CORDOVA RIOJA - 750ML,WINE,0.0,4.0,0.0


In [15]:
engine = create_engine('mysql+pymysql://Svetlana Gruzdeva:***@localhost/warehouse')
# I removed my real password and saved samples of sql tables below as a proof that my command did work.
clean_data.to_sql(con=engine, name='cleaned_data', if_exists='replace')
data = pd.read_sql_query('''SELECT * FROM cleaned_data LIMIT 10''', engine)
data

,index,YEAR,MONTH,SUPPLIER,ITEM CODE,ITEM DESCRIPTION,ITEM TYPE,RETAIL SALES,RETAIL TRANSFERS,WAREHOUSE SALES
0,0,2017,4,ROYAL WINE CORP,100200,GAMLA CAB - 750ML,WINE,0.0,1.0,0.0
1,1,2017,4,SANTA MARGHERITA USA INC,100749,SANTA MARGHERITA P/GRIG ALTO - 375ML,WINE,0.0,1.0,0.0
2,2,2017,4,JIM BEAM BRANDS CO,10103,KNOB CREEK BOURBON 9YR - 100P - 375ML,LIQUOR,0.0,8.0,0.0
3,3,2017,4,HEAVEN HILL DISTILLERIES INC,10120,J W DANT BOURBON 100P - 1.75L,LIQUOR,0.0,2.0,0.0
4,4,2017,4,ROYAL WINE CORP,101664,RAMON CORDOVA RIOJA - 750ML,WINE,0.0,4.0,0.0
5,5,2017,4,REPUBLIC NATIONAL DISTRIBUTING CO,101680,MANISCHEWITZ CREAM WH CONCORD - 1.5L,WINE,0.0,1.0,0.0
6,6,2017,4,ROYAL WINE CORP,101753,BARKAN CLASSIC PET SYR - 750ML,WINE,0.0,1.0,0.0
7,7,2017,4,JIM BEAM BRANDS CO,10197,KNOB CREEK BOURBON 9YR - 100P - 1.75L,LIQUOR,0.0,32.0,0.0
8,8,2017,4,STE MICHELLE WINE ESTATES,101974,CH ST MICH P/GRIS - 750ML,WINE,0.0,26.0,0.0
9,9,2017,4,MONSIEUR TOUTON SELECTION,102083,CH DE LA CHESNAIE MUSCADET - 750ML,WINE,0.0,1.0,0.0


In [16]:
col = ['RETAIL SALES', 'RETAIL TRANSFERS', 'WAREHOUSE SALES']
agg_supplier = clean_data.groupby('SUPPLIER')[col].sum().reset_index()
agg_supplier.to_sql(con=engine, name='aggregated_per_supplier', if_exists='replace')
data = pd.read_sql_query('''SELECT * FROM aggregated_per_supplier LIMIT 10''', engine)
data

,index,SUPPLIER,RETAIL SALES,RETAIL TRANSFERS,WAREHOUSE SALES
0,0,8 VINI INC,2.78,2.00,1.00
1,1,A HARDY USA LTD,0.40,0.00,0.00
2,2,A I G WINE & SPIRITS,12.52,5.92,134.00
3,3,A VINTNERS SELECTIONS,8640.57,8361.10,29776.67
4,4,A&E INC,11.52,2.00,0.00
5,5,A&W BORDERS LLC,0.80,1.00,0.00
6,6,ADAMBA IMPORTS INTL,32.20,40.49,0.00
7,7,AIKO IMPORTERS INC,11.24,11.00,3.00
8,8,ALLAGASH BREWING COMPANY,304.09,339.00,1742.92
9,9,ALLIED IMPORTERS USA LTD,7.63,11.00,18.00


In [17]:
agg_item = clean_data.groupby('ITEM CODE')[col].sum().reset_index()
agg_item.to_sql(con=engine, name='aggregated_per_item', if_exists='replace')
data = pd.read_sql_query('''SELECT * FROM aggregated_per_item LIMIT 10''', engine)
data

,index,ITEM CODE,RETAIL SALES,RETAIL TRANSFERS,WAREHOUSE SALES
0,0,100003,0.0,0.0,1.0
1,1,100007,0.0,0.0,1.0
2,2,100008,0.0,0.0,1.0
3,3,100009,0.0,0.0,12.0
4,4,100011,0.0,0.0,3.0
5,5,100012,0.0,0.0,17.0
6,6,10004,0.0,0.0,1.0
7,7,100080,0.0,0.0,6.0
8,8,1001,0.0,0.0,9.0
9,9,10014,1.0,1.0,13.0
